In [ ]:
%cd ..

In [2]:
from typing import Union
from tqdm import tqdm
from collections import deque
from statistics import mean
from functools import partial

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision.models import alexnet, AlexNet_Weights

from conv_cp.conv_cp import decompose_model
from conv_cp.imagenet.dataset import ImageNet

In [3]:
def acc_fn_1(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = y_pred.argmax(dim=1)
    correct = (y_pred == y_true).sum().item()
    return correct / y_true.size(0)


def acc_fn_5(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = y_pred.topk(5, dim=1).indices
    correct = (y_pred == y_true.unsqueeze(1)).sum().item()
    return correct / y_true.size(0)


def loss_fn(
    model: nn.Module,
    dataloader: DataLoader,
    device: Union[str, torch.device],
    verbose: bool = False,
    num_steps: int = 50,
) -> float:
    model.eval()
    model.to(device)

    total_loss = 0
    data_iter = iter(dataloader)
    loop = range(num_steps)
    if verbose:
        loop = tqdm(range(num_steps), desc="Validation")
    for step in loop:
        try:
            x, y = next(data_iter)
        except StopIteration:
            break
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            y_pred = model(x)
            loss = F.cross_entropy(y_pred, y)
            total_loss += loss.item()

        if verbose:
            loop.set_postfix(loss=total_loss / (step + 1))

    model.cpu()
    return total_loss / num_steps


def train_fn(
    model: nn.Module,
    dataloader: DataLoader,
    device: Union[str, torch.device],
    lr: float,
    num_steps: int = 1000,
    metric_len: int = 20,
    loss_tol: float = 1e-3,
    acc_tol: float = 0.95,
    verbose: bool = False,
):
    model.train()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    running_loss = deque(maxlen=metric_len)
    running_acc = deque(maxlen=metric_len)
    runngin_acc_5 = deque(maxlen=metric_len)

    data_iter = iter(dataloader)
    loop = range(num_steps)
    if verbose:
        loop = tqdm(range(num_steps), desc="Training")
    for _ in loop:
        try:
            x, y = next(data_iter)
        except StopIteration:
            data_iter = iter(dataloader)
            x, y = next(data_iter)

        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())
        running_acc.append(acc_fn_1(y_pred, y))
        runngin_acc_5.append(acc_fn_5(y_pred, y))

        if verbose:
            loop.set_postfix(
                loss=mean(running_loss),
                acc_1=mean(running_acc),
                acc_5=mean(runngin_acc_5),
            )

        if mean(running_loss) < loss_tol or mean(running_acc) > acc_tol:
            break

    optimizer.zero_grad()
    model.cpu()
    model.eval()

In [4]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.cuda()
transform = AlexNet_Weights.IMAGENET1K_V1.transforms()

dataset = ImageNet(root_dir="data/val-images", transform=transform)
train_dataset, val_dataset = dataset.split(0.9)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

In [5]:
loss_fn = partial(
    loss_fn,
    dataloader=val_loader,
    device="cuda",
    num_steps=20,
    verbose=True,
)
train_fn = partial(
    train_fn,
    dataloader=train_loader,
    device="cuda",
    lr=1e-7,
    metric_len=10,
    num_steps=100,
    verbose=True,
)

In [6]:
cp_model = decompose_model(
    model,
    conv_rank=2000,
    fc_rank=1500,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@2000@1500.pt")
del cp_model

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it, loss=8.32]


Processing module features.3


Validation: 100%|██████████| 20/20 [00:28<00:00,  1.45s/it, loss=7.47]


Processing module features.6


...

Decomposing classifier.1 with rank 704
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [02:48<00:00,  1.69s/it, acc_1=0.526, acc_5=0.755, loss=2.13]


Decomposing classifier.4 with rank 487
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [02:55<00:00,  1.76s/it, acc_1=0.52, acc_5=0.754, loss=2.16]


Decomposing classifier.6 with rank 309


In [8]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.cuda()
cp_model = decompose_model(
    model,
    conv_rank=1800,
    fc_rank=1400,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@1800@1400.pt")
del cp_model

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it, loss=8.32]


Processing module features.3


Validation: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it, loss=7.47]


Processing module features.6


...

Decomposing classifier.1 with rank 657
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [02:48<00:00,  1.69s/it, acc_1=0.507, acc_5=0.739, loss=2.2]


Decomposing classifier.4 with rank 454
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [02:57<00:00,  1.77s/it, acc_1=0.486, acc_5=0.741, loss=2.25]


Decomposing classifier.6 with rank 289


In [9]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.cuda()
cp_model = decompose_model(
    model,
    conv_rank=1600,
    fc_rank=1350,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@1600@1350.pt")
del cp_model

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 20/20 [00:31<00:00,  1.57s/it, loss=8.32]


Processing module features.3


Validation: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it, loss=7.47]


Processing module features.6


...

Decomposing classifier.1 with rank 633
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [02:53<00:00,  1.73s/it, acc_1=0.5, acc_5=0.743, loss=2.22] 


Decomposing classifier.4 with rank 439
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [02:50<00:00,  1.71s/it, acc_1=0.508, acc_5=0.747, loss=2.22]


Decomposing classifier.6 with rank 278


In [10]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.cuda()
cp_model = decompose_model(
    model,
    conv_rank=1400,
    fc_rank=1250,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@1400@1250.pt")
del cp_model

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 20/20 [00:32<00:00,  1.64s/it, loss=9.24]


Processing module features.3


Validation: 100%|██████████| 20/20 [00:27<00:00,  1.40s/it, loss=7.48]


Processing module features.6


...

Decomposing classifier.1 with rank 586
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [02:50<00:00,  1.70s/it, acc_1=0.507, acc_5=0.762, loss=2.12]


Decomposing classifier.4 with rank 406
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [02:46<00:00,  1.66s/it, acc_1=0.527, acc_5=0.761, loss=2.12]


Decomposing classifier.6 with rank 258


In [11]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.cuda()
cp_model = decompose_model(
    model,
    conv_rank=1200,
    fc_rank=1150,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@1200@1150.pt")
del cp_model

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 20/20 [00:30<00:00,  1.52s/it, loss=9.25]


Processing module features.3


Validation: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it, loss=7.47]


Processing module features.6


...

Decomposing classifier.1 with rank 540
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [02:43<00:00,  1.64s/it, acc_1=0.483, acc_5=0.735, loss=2.33]


Decomposing classifier.4 with rank 373
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [02:40<00:00,  1.60s/it, acc_1=0.489, acc_5=0.739, loss=2.26]


Decomposing classifier.6 with rank 237


In [ ]:
cp_model = decompose_model(
    model,
    conv_rank=1000,  
    fc_rank=1050,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@1000@1050.pt")
del cp_model

In [12]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.cuda()
cp_model = decompose_model(
    model,
    conv_rank=900,
    fc_rank=950,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.8,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)
cp_model = cp_model.cpu()
torch.save(cp_model, "checkpoints/model@900@950.pt")
del cp_model

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 20/20 [00:28<00:00,  1.44s/it, loss=8.32]


Processing module features.3


Validation: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it, loss=7.48]


Processing module features.6


...

Decomposing classifier.1 with rank 446
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [02:46<00:00,  1.66s/it, acc_1=0.477, acc_5=0.73, loss=2.31]


Decomposing classifier.4 with rank 308
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [02:39<00:00,  1.60s/it, acc_1=0.478, acc_5=0.737, loss=2.31]


Decomposing classifier.6 with rank 196
